In [48]:
# Code you have previously used to load data
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
#from learntools.core import *

ImportError: cannot import name 'cross_validate' from 'sklearn' (/Users/ccc/src/checkout/anaconda3/lib/python3.7/site-packages/sklearn/__init__.py)

In [30]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [32]:
def get_max_leaf_nodes(train_X, val_X, train_y, val_y):
    test_leaf_nodes = [50, 90, 100, 110, 150, 500]
    min = get_mae(test_leaf_nodes[0], train_X, val_X, train_y, val_y)
    for i in test_leaf_nodes:
        my_mae = get_mae(i, train_X, val_X, train_y, val_y)
        if (min > my_mae):
            max_leaf_nodes = i
            min = my_mae
    return(max_leaf_nodes)

In [33]:
def read_training_data():
    iowa_file_path = '../data/train.csv'
    home_data = pd.read_csv(iowa_file_path)
    y = home_data.SalePrice
    features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
    X = home_data[features]
    train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
    return(X, train_X, val_X, y, train_y, val_y, features)

In [49]:
def cv_loop(X, y, model, K):
    '''
    Cross validation: for K iterations, split the data into train and test
    sets, build a model, and return the mean AUC.
    '''
    
    SEED = 15
    mean_auc = 0.
    for i in range(K):
        X_train, X_cv, y_train, y_cv = model_selection.train_test_split(
            X, y, test_size = 0.2,
            random_state = i*SEED)
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_cv)[:,1]
        auc = metrics.roc_auc_score(y_cv, preds)
        print("AUC (fold %d/%d): %f" % (i + 1, K, auc))
        mean_auc += auc
    return mean_auc/K

In [34]:
def create_submission(features):
    test_data_path = '../data/test.csv'
    test_data = pd.read_csv(test_data_path)
    test_X = test_data[features]
    # make predictions which we will submit.
    test_preds = rf_model_on_full_data.predict(test_X)
    # save predictions in format used for competition scoring
    output = pd.DataFrame({'Id': test_data.Id,'SalePrice': test_preds})
    output.to_csv('submission.csv', index=False)

## Organize Data

In [35]:
X, train_X, val_X, y, train_y, val_y, features = read_training_data()

## Decision Tree

In [41]:
my_max_leaf_nodes=get_max_leaf_nodes(train_X, val_X, train_y, val_y)
iowa_model = DecisionTreeRegressor(max_leaf_nodes=my_max_leaf_nodes, random_state=1)

iowa_model.fit(train_X, train_y)
val_predictions = iowa_model.predict(val_X)

val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE for best value of max_leaf_nodes: ${:,.0f}".format(val_mae))

Validation MAE for best value of max_leaf_nodes: $27,283


## Random Forest

In [22]:
rf_model = RandomForestRegressor(random_state=1)

rf_model.fit(train_X, train_y)
rf_val_predictions = rf_model.predict(val_X)

rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)
print("Validation MAE for Random Forest Model: ${:,.0f}".format(rf_val_mae))

Validation MAE for Random Forest Model: $22,762


/Users/ccc/src/checkout/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [23]:
# To improve accuracy, create a new Random Forest model which you will train on all training data
rf_model_on_full_data = RandomForestRegressor(random_state=1)

# fit rf_model_on_full_data on all data from the training data
rf_model_on_full_data.fit(X, y)

/Users/ccc/src/checkout/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

## Gaussian NB

In [50]:
model = GaussianNB()
K = 10 # Number of iterations for cross validation

print("AUC score for 10 fold cross validation:\n")
score = cv_loop(train_X, train_y, model, K)

print("\nMean AUC: %f" %score)

AUC score for 10 fold cross validation:



NameError: name 'model_selection' is not defined

In [ ]:
# check submission directory for CSV file
create_submission(features)